# LNR - Práctica 3
## Sesión 4: Combinación de Clasificadores

Nombre:
- Guillermo Ferrando Muñoz

Ejercicio:

Implementar y evaluar la combinación modelos de clasificación en la tarea DETESTS (al
menos dos técnicas diferentes).

## Carga de librerías y datos

De nuevo, cargamos los datos con el desbalanceo solucionado mediante SMOTE oversampling que obtuvimos en la sesión 2 de esta práctica. Para ello, leemos el documento "S3_P3.csv" que he generado, lo incluiré en la entrega de la tarea.

In [62]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn import metrics
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.model_selection import cross_val_score

In [2]:
df=pd.read_csv("S3_P3.csv")
df=df.drop(columns=['Unnamed: 0']) #para eliminar una columna inútil
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,stereotype
0,0.001318,0.003393,0.002340,-0.002296,0.004027,0.002442,0.000703,0.001406,-0.002482,0.000745,...,-0.001195,-0.000311,-0.000729,0.001500,-0.000136,-0.002057,0.004157,0.002924,0.003697,0
1,0.001163,0.002280,0.005396,-0.005706,0.003198,0.001721,0.002030,0.004187,-0.003248,0.007304,...,-0.002858,0.007483,-0.001480,0.005370,-0.002683,0.005325,-0.001532,0.000768,-0.002241,0
2,-0.001684,0.003351,-0.000300,0.000164,0.000150,-0.003695,0.001025,0.005774,0.001311,-0.000973,...,0.000300,0.000582,-0.004117,0.001826,0.002370,0.000677,-0.001318,-0.000238,0.002707,0
3,0.001581,-0.000432,0.002926,-0.001243,0.000831,-0.002557,0.003618,0.003611,0.000520,-0.000479,...,0.001939,-0.002411,-0.002368,0.001319,-0.000663,0.004117,-0.000327,0.003946,0.004706,0
4,-0.000496,-0.000016,0.007480,-0.002841,-0.001257,-0.001341,0.001828,0.003550,0.003153,0.001166,...,0.000466,0.005146,0.002355,0.002462,0.003463,0.002679,0.001108,-0.001922,0.000507,0
5,0.000717,0.001005,-0.000263,0.000679,0.001691,-0.003206,-0.002786,0.006954,-0.001253,0.001588,...,0.000191,-0.002558,-0.002527,0.007731,0.003119,-0.003920,0.001947,-0.004730,-0.002866,0
6,-0.004574,-0.001306,0.000020,-0.002135,-0.000803,0.000261,-0.000666,0.002697,0.000333,0.002623,...,-0.002448,0.001542,0.000938,0.005675,-0.000728,0.002071,-0.001132,0.002149,-0.001617,0
7,-0.007377,0.000183,0.004989,-0.006015,0.002102,-0.002705,-0.006664,0.009896,-0.009904,0.007932,...,0.008981,0.001597,0.000005,-0.006048,0.009836,0.006450,-0.005930,0.005494,0.005669,0
8,-0.000654,0.001425,0.000660,0.001815,-0.002990,-0.001975,0.000159,0.004599,-0.004364,0.000639,...,-0.003120,0.004801,-0.000113,0.003341,0.002509,0.002654,0.000550,0.001538,0.003378,0
9,0.004029,0.002029,0.004909,-0.000626,0.000693,-0.003865,0.001327,-0.008393,-0.000097,-0.005665,...,0.001387,0.005404,-0.005205,0.001415,0.006878,0.000329,-0.003058,-0.003888,0.002836,0


In [64]:
X = df.drop("stereotype", axis=1).values
y = df["stereotype"].values

In [65]:
X, y = shuffle(X, y) #para barajar los datos

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                          test_size=0.25, shuffle = True, random_state=42) #shuffle = True para barajar los datos

## Breve explicación inicial

Vamos a implementar combinaciones de modelos. Aprovecharemos lo que estudiamos en la anterior sesión con GridSearchCV y RandomizedSearchCV, utilizando los modelos óptimos de cada tipo de clasificador para nuestros datos.

## Bagging

### Regresión logística

In [67]:
import time

tini = time.process_time()

#X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                          test_size=0.25, shuffle = True, random_state=42)

clf = BaggingClassifier(base_estimator=LogisticRegression(C = 0.7839693033701929, penalty = 'l2'),n_estimators=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, zero_division=1)
print("F1", a)
a=accuracy_score(y_test, y_pred, normalize=True)
print("Accuracy", a)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 0.5880721220527045
Accuracy 0.5967413441955194

Ha tardado 0.92 segundos


Mismas puntuaciones que en la sesión anterior, aunque quizás F1 mejora un poco. Vamos a hacer validación cruzada para ver si ha mejorado.

In [68]:
import time
from sklearn.model_selection import KFold

tini = time.process_time()

for i in range(3,11):
    kf = KFold(n_splits=i)
    print('K =',i)    
    f=[]
    mac=[]
    mic=[]
    for train, test in kf.split(X):
        X_train_i, X_test_i, y_train_i, y_test_i = X[train], X[test], y[train], y[test]

        clf = BaggingClassifier(base_estimator=LogisticRegression(C = 0.7839693033701929, penalty = 'l2'),
                                n_estimators=10).fit(X_train_i, y_train_i)
        
        y_pred_i = clf.predict(X_test_i)
        
        f1=f1_score(y_test_i, y_pred_i, zero_division=1)
        f.append(f1)

        macro=f1_score(y_test_i, y_pred_i, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                        #This does not take label imbalance into account.
        mac.append(macro)

        micro=f1_score(y_test_i, y_pred_i, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                        #false negatives and false positives.
        mic.append(micro)
        
    print("F1 promedio", sum(f)/i)
    print("macro F1 promedio", sum(mac)/i)
    print("micro F1 promedio", sum(mic)/i)
    print()


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

K = 3
F1 promedio 0.4509813075815834
macro F1 promedio 0.4803020917946812
micro F1 promedio 0.5453156822810591

K = 4
F1 promedio 0.4100753614665429
macro F1 promedio 0.45281105636838936
micro F1 promedio 0.5271554650373387

K = 5
F1 promedio 0.5792877151918434
macro F1 promedio 0.5483528069732077
micro F1 promedio 0.5719596331384975

K = 6
F1 promedio 0.5276445730444462
macro F1 promedio 0.5041408795425597
micro F1 promedio 0.5521045485403938

K = 7
F1 promedio 0.5440827888514423
macro F1 promedio 0.5501146310863597
micro F1 promedio 0.5736588166607287

K = 8
F1 promedio 0.5314574242202988
macro F1 promedio 0.5643947970667003
micro F1 promedio 0.5775629479824199

K = 9
F1 promedio 0.5599483864520995
macro F1 promedio 0.5457210403981444
micro F1 promedio 0.56991411889514

K = 10
F1 promedio 0.5478776938574172
macro F1 promedio 0.5513108863241392
micro F1 promedio 0.5719553394146931


Ha tardado 70.14 segundos


Parece que era una falsa alarma.

### Árboles de decisión

In [69]:
import time

tini = time.process_time()

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                          test_size=0.25, shuffle = True, random_state=42)

clf=BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion = 'gini', max_depth=30),n_estimators=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, zero_division=1)
print("F1", a)
a=accuracy_score(y_test, y_pred, normalize=True)
print("Accuracy", a)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 0.8210227272727272
Accuracy 0.8289205702647657

Ha tardado 9.08 segundos


Estos resultados suponen una notable mejora respecto a lo que obtuvimos en la sesión anterior. Para comprobar que no hemos sobreajustado, vamos a aplicar k-fold.

In [70]:
import time
from sklearn.model_selection import KFold

tini = time.process_time()

for i in range(3,11):
    kf = KFold(n_splits=i)
    print('K =',i)    
    f=[]
    mac=[]
    mic=[]
    for train, test in kf.split(X):
        X_train_i, X_test_i, y_train_i, y_test_i = X[train], X[test], y[train], y[test]

        clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion = 'gini', max_depth=30), 
                                n_estimators=10).fit(X_train_i, y_train_i)
        y_pred_i = clf.predict(X_test_i)
        
        f1=f1_score(y_test_i, y_pred_i, zero_division=1)
        f.append(f1)

        macro=f1_score(y_test_i, y_pred_i, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                        #This does not take label imbalance into account.
        mac.append(macro)

        micro=f1_score(y_test_i, y_pred_i, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                        #false negatives and false positives.
        mic.append(micro)
        
    print("F1 promedio", sum(f)/i)
    print("macro F1 promedio", sum(mac)/i)
    print("micro F1 promedio", sum(mic)/i)
    print()


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

K = 3
F1 promedio 0.8014695045630376
macro F1 promedio 0.8099980934431711
micro F1 promedio 0.8104209097080788

K = 4
F1 promedio 0.8119499223730562
macro F1 promedio 0.8205114612815552
micro F1 promedio 0.8209436524100475

K = 5
F1 promedio 0.8150545304904074
macro F1 promedio 0.8236462982454544
micro F1 promedio 0.8241681318950335

K = 6
F1 promedio 0.8289298493296963
macro F1 promedio 0.8348433339110196
micro F1 promedio 0.8352002715546503

K = 7
F1 promedio 0.8242092870360084
macro F1 promedio 0.8314172389160285
micro F1 promedio 0.8318003717680611

K = 8
F1 promedio 0.8187111872811541
macro F1 promedio 0.8269312786605031
micro F1 promedio 0.8273878108223705

K = 9
F1 promedio 0.8290954372210428
macro F1 promedio 0.835816815870529
micro F1 promedio 0.8362137612085088

K = 10
F1 promedio 0.8350642233758971
macro F1 promedio 0.8414565925947308
micro F1 promedio 0.8418189404621451


Ha tardado 547.08 segundos


En efecto, el ajuste es bueno. Eso sí, el coste temporal aumenta mucho.

### Random Forest

**Nota:** Random Forest no lo habíamos implementado en las anteriores sesiones, lo hago ahora por incluir algo nuevo.

**Nota 2:** He intentado sacar un modelo óptimo con GridSearchCV, pero no ha sido la mejor idea (tardaba muchísimo). Lo usaremos con los valores por defecto.

In [71]:
import time

tini = time.process_time()

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                          test_size=0.25, shuffle = True, random_state=42)

rf_clf=BaggingClassifier(base_estimator=RandomForestClassifier(),n_estimators=10)
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)

a=f1_score(y_test, y_pred, zero_division=1)
print("F1", a)
a=accuracy_score(y_test, y_pred, normalize=True)
print("Accuracy", a)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 0.837916063675832
Accuracy 0.8479293957909029

Ha tardado 57.8 segundos


Comprobamos de nuevo con k-fold:

In [80]:
### (TARDA)
import time
from sklearn.model_selection import KFold

tini = time.process_time()

for i in range(3,11):
    kf = KFold(n_splits=i)
    print('K =',i)    
    f=[]
    mac=[]
    mic=[]
    for train, test in kf.split(X):
        X_train_i, X_test_i, y_train_i, y_test_i = X[train], X[test], y[train], y[test]

        clf = BaggingClassifier(base_estimator=RandomForestClassifier(), 
                                n_estimators=10).fit(X_train_i, y_train_i)
        y_pred_i = clf.predict(X_test_i)
        
        f1=f1_score(y_test_i, y_pred_i, zero_division=1)
        f.append(f1)

        macro=f1_score(y_test_i, y_pred_i, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                        #This does not take label imbalance into account.
        mac.append(macro)

        micro=f1_score(y_test_i, y_pred_i, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                        #false negatives and false positives.
        mic.append(micro)
        
    print("F1 promedio", sum(f)/i)
    print("macro F1 promedio", sum(mac)/i)
    print("micro F1 promedio", sum(mic)/i)
    print()


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

K = 3
F1 promedio 0.8353435868865016
macro F1 promedio 0.8434171906278277
micro F1 promedio 0.8438560760353021

K = 4
F1 promedio 0.8391633166848141
macro F1 promedio 0.8468544974455183
micro F1 promedio 0.8472505091649695

K = 5
F1 promedio 0.8470883442392205
macro F1 promedio 0.8538297160871435
micro F1 promedio 0.8542085534775954

K = 6


KeyboardInterrupt: 

Parece que sí ha ajustado bien.

### SVM

In [72]:
import time

tini = time.process_time()

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                          test_size=0.25, shuffle = True, random_state=42)

clf=BaggingClassifier(base_estimator=svm.SVC(kernel='rbf', C=10),n_estimators=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, zero_division=1)
print("F1", a)
a=accuracy_score(y_test, y_pred, normalize=True)
print("Accuracy", a)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 0.8688200395517468
Accuracy 0.8649015614392397

Ha tardado 23.23 segundos


Comprobación con k-fold:

In [73]:
import time
from sklearn.model_selection import KFold

tini = time.process_time()

for i in range(3,11):
    kf = KFold(n_splits=i)
    print('K =',i)    
    f=[]
    mac=[]
    mic=[]
    for train, test in kf.split(X):
        X_train_i, X_test_i, y_train_i, y_test_i = X[train], X[test], y[train], y[test]

        clf = BaggingClassifier(base_estimator=svm.SVC(kernel='rbf', C=10),
                                n_estimators=10).fit(X_train_i, y_train_i)
        y_pred_i = clf.predict(X_test_i)
        
        f1=f1_score(y_test_i, y_pred_i, zero_division=1)
        f.append(f1)

        macro=f1_score(y_test_i, y_pred_i, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                        #This does not take label imbalance into account.
        mac.append(macro)

        micro=f1_score(y_test_i, y_pred_i, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                        #false negatives and false positives.
        mic.append(micro)
    print("F1 promedio", sum(f)/i)
    print("macro F1 promedio", sum(mac)/i)
    print("micro F1 promedio", sum(mic)/i)
    print()


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

K = 3
F1 promedio 0.8664899180713643
macro F1 promedio 0.8612469086291924
micro F1 promedio 0.8615071283095723

K = 4
F1 promedio 0.8724550773945486
macro F1 promedio 0.8663030901397231
micro F1 promedio 0.8665987780040734

K = 5
F1 promedio 0.8823258651447403
macro F1 promedio 0.8764466816047725
micro F1 promedio 0.876781854496703

K = 6
F1 promedio 0.8873787768908311
macro F1 promedio 0.8818424815218603
micro F1 promedio 0.882213170400543

K = 7
F1 promedio 0.8841636690018005
macro F1 promedio 0.8784832981942589
micro F1 promedio 0.8788180971236998

K = 8
F1 promedio 0.8849337777119755
macro F1 promedio 0.8793592405106837
micro F1 promedio 0.8796676357589522

K = 9


KeyboardInterrupt: 

Sorprendentemente, con Bagging, árboles de decisión mejora su puntación de la anterior sesión alrededor de un 10%, aún así, Random Forest acaba siendo ligeramente mejor. No sin mejorar la puntuación del modelo SVM que obtuvimos en la anterior sesión de la práctica gracias a GridSearchCV, que aumenta los aciertos al emplearlo con Bagging.

## Adaboost

### Regresión logística

In [75]:
import time

tini = time.process_time()

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                          test_size=0.25, shuffle = True, random_state=42)

clf=AdaBoostClassifier(base_estimator=LogisticRegression(C = 0.7839693033701929, penalty = 'l2'),n_estimators=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, zero_division=1)
print("F1", a)
a=accuracy_score(y_test, y_pred, normalize=True)
print("Accuracy", a)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 0.6566347469220246
Accuracy 0.48879837067209775

Ha tardado 0.27 segundos


### Árboles de decisión

In [76]:
import time
from sklearn.tree import DecisionTreeClassifier

tini = time.process_time()

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                          test_size=0.25, shuffle = True, random_state=42)

clf=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion = 'gini', max_depth=30),n_estimators=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, zero_division=1)
print("F1", a)
a=accuracy_score(y_test, y_pred, normalize=True)
print("Accuracy", a)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 0.8021462105969148
Accuracy 0.7997284453496266

Ha tardado 17.45 segundos


### Random Forest

In [77]:
import time

tini = time.process_time()

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                          test_size=0.25, shuffle = True, random_state=42)

clf=AdaBoostClassifier(base_estimator=RandomForestClassifier(),n_estimators=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, zero_division=1)
print("F1", a)
a=accuracy_score(y_test, y_pred, normalize=True)
print("Accuracy", a)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 0.8503532434168273
Accuracy 0.8418194161575017

Ha tardado 112.36 segundos


Parece que mejora un poco en comparación con el de Bagging, vamos a comprobarlo.

In [81]:
### (TARDA)
import time
from sklearn.model_selection import KFold

tini = time.process_time()

for i in range(3,11):
    kf = KFold(n_splits=i)
    print('K =',i)    
    f=[]
    mac=[]
    mic=[]
    for train, test in kf.split(X):
        X_train_i, X_test_i, y_train_i, y_test_i = X[train], X[test], y[train], y[test]

        clf = AdaBoostClassifier(base_estimator=RandomForestClassifier(),n_estimators=10).fit(X_train_i, y_train_i)
        y_pred_i = clf.predict(X_test_i)
        
        f1=f1_score(y_test_i, y_pred_i, zero_division=1)
        f.append(f1)

        macro=f1_score(y_test_i, y_pred_i, average='macro') #Calculate metrics for each label, and find their unweighted mean. 
                                                        #This does not take label imbalance into account.
        mac.append(macro)

        micro=f1_score(y_test_i, y_pred_i, average='micro') #Calculate metrics globally by counting the total true positives, 
                                                        #false negatives and false positives.
        mic.append(micro)
        
    print("F1 promedio", sum(f)/i)
    print("macro F1 promedio", sum(mac)/i)
    print("micro F1 promedio", sum(mic)/i)
    print()


tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

K = 3
F1 promedio 0.8530618371280708
macro F1 promedio 0.848869946384253
micro F1 promedio 0.8494568906992533

K = 4
F1 promedio 0.8583858072292216
macro F1 promedio 0.8493935823086431
micro F1 promedio 0.8499660556687034

K = 5
F1 promedio 0.8669438601525581
macro F1 promedio 0.859719847545988
micro F1 promedio 0.8601498205005249

K = 6


KeyboardInterrupt: 

Tarda mucho, he parado la ejecución del bucle. Por los folds aplicados, los resultados parecen similares al mismo modelo aplicado con Bagging. Igualmente, no parece que los resultados vayan a superar a los del modelo óptimo de SVM, pero sí se quedarán muy muy cerca.

### SVM

In [63]:
import time

tini = time.process_time()

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                          test_size=0.25, shuffle = True, random_state=42)

clf=AdaBoostClassifier(base_estimator=svm.SVC(kernel='rbf', C=10),n_estimators=10, algorithm='SAMME')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, zero_division=1)
print("F1", a)
a=accuracy_score(y_test, y_pred, normalize=True)
print("Accuracy", a)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

F1 0.3554555680539932
Accuracy 0.6109979633401222

Ha tardado 89.23 segundos


Interesante.

## Stacking

### Red neuronal y SVM con Regresión Logística de metaclasificador

In [78]:
import time

tini = time.process_time()
estimators=[('mlp',MLPClassifier(random_state=1, max_iter=300)),('svr', svm.SVC(kernel='rbf', C=10))]
clf=StackingClassifier(estimators=estimators,final_estimator=LogisticRegression(C = 0.7839693033701929, penalty = 'l2'))
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

a=f1_score(y_test, y_pred, zero_division=1)
print("F1", a)
a=accuracy_score(y_test, y_pred, normalize=True)
print("Accuracy", a)

tfin = time.process_time()
print()
print("Ha tardado", round(tfin-tini, 2), "segundos")

C:\Users\guill\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\guill\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\guill\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\guill\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\guill\Anaconda3\lib\site-packages\sklearn\n

F1 0.8938471940500338
Accuracy 0.8934147997284454

Ha tardado 287.84 segundos


Parece que tenemos un nuevo campeón.

## Breve conclusión

El mejor clasificador pasa a ser ahora el empleado con Stacking, seguido de SVM aplicado con Bagging, aunque los otros tipos de clasificadores empiezan a pisarles los talones gracias a las distintas técnicas de combinación de modelos.